# Merging Landsat Bands into a Single Multiband Image

This notebook takes in a folder of freshly unzipped Landsat images in GeoTIFF format and merge them into a 
single multiband image. This completes the task by first identifying all the bands you want to combine, then
reads in each separate band as a numpy array. After each separate band is read and stored into a list, the 
geotransform and projection information is copied over via GDAL from the original input image for the output.
Lastly, the band-specific arrays are written to the output dataset as separate bands. 

The only inputs you need in this notebook are to change the **in_tif_dir** variable and an output file name with 
a *.tif* extension. You will be prompted to select which bands you want to stack. I will eventually add support 
for previous Landsat satellites as the band reference information differs from platform to platform. 

In [1]:
import numpy as np
from osgeo import gdal, ogr, osr
import matplotlib.pyplot as plt
import os

from IPython.display import Javascript, display
from ipywidgets import widgets

%matplotlib inline

I suggest putting your raw landsat unzipped data in a folder titled 'data' one directory above
the 'scripts' directory, as the os.walk function in the next cell will look through a certain folder
for landsat images ending with the specified band combos. See this link for more info:
https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites?qt-news_science_products=0


In [2]:
in_tif_dir = r'../../data'
out_tif_name = 'Austin_Landsat.tif'
tif_driver = gdal.GetDriverByName('GTiff')
out_tif_path = os.path.join(in_tif_dir, 
                            out_tif_name)

In [3]:
def run_next_cell(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

Preset dictionary with the band designations. From the selector, choose which bands you want to include.

Note: This is only for Landsat 8 band configurations

In [4]:
band_reference_dict = {'Coastal Aerosol': 1, 
                       'Blue': 2, 
                       'Green': 3, 
                       'Red': 4, 
                       'NIR': 5, 
                       'SWIR 1': 6, 
                       'SWIR 2': 7, 
                       'Panchromatic': 8,
                       'Cirrus': 9, 
                       'TIRS 1': 10, 
                       'TIRS 2': 11}

selector = widgets.SelectMultiple(options=band_reference_dict.keys())
button = widgets.Button(description='Select these bands')
button.on_click(run_next_cell)
display(selector)
display(button)

SelectMultiple(options=('Coastal Aerosol', 'Blue', 'Green', 'Red', 'NIR', 'SWIR 1', 'SWIR 2', 'Panchromatic', …

Button(description='Select these bands', style=ButtonStyle())

<IPython.core.display.Javascript object>

In [5]:
print("You have chosen to merge the following bands: {}\n Please run the next cell".format(selector.value))
band_reference_list = [band_reference_dict[x] for x in selector.value]

You have chosen to merge the following bands: ('Blue', 'Green', 'Red', 'NIR')
 Please run the next cell


This next cell finds all the appropriate GeoTIFF images in your in directory, opens up in GDAL, reads them as 
numpy arrays, and stores in a list.

In [7]:
tif_dir_list = []

for dirs, subdirs, files in os.walk(in_tif_dir):
    for file in files:
        fname = file.split('.')[0]
        ext = file.split('.')[-1].lower()
        if ext == 'tif':
            for band_num in band_reference_list:
                if fname.endswith('B{}'.format(band_num)):
                    path = os.path.join(dirs, file)
                    ds = gdal.Open(path)
                    arr = ds.GetRasterBand(1).ReadAsArray()       
                    tif_dir_list.append(arr)

First open any of the images then stores the geotransform and projection info

In [8]:
in_ds = gdal.Open(path)
in_gt = in_ds.GetGeoTransform()
prj = in_ds.GetProjection()
srs = osr.SpatialReference(wkt=prj)

Creates an out tif with the info you provided in cell 2 as well as 
predetermined variables projection and geotransform

In [9]:
out_tif = tif_driver.Create(out_tif_path, 
                            tif_dir_list[0].shape[1],
                            tif_dir_list[0].shape[0],
                            len(tif_dir_list),
                            gdal.GDT_Float64)

out_tif.SetProjection(srs.ExportToWkt())
out_tif.SetGeoTransform(in_gt);

GDAL suxx and starts indexing at 1, instead of good ole 0.

So we loop through each stored array in the list and write it as a separate color band

In [10]:
i = 1
for arr in tif_dir_list:    
    out_band = out_tif.GetRasterBand(i)
    out_band.WriteArray(arr)
    i+=1
out_tif.FlushCache()
del out_tif